# Creating and Valuing a CDS Contract

Replicating Markit Pricing on 20 Aug 2020

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.utils import *
from financepy.products.rates import *
from financepy.products.credit import *

####################################################################
# FINANCEPY BETA Version 0.33 - This build:  11 Nov 2023 at 07:47 #
#     This software is distributed FREE AND WITHOUT ANY WARRANTY   #
#  Report bugs as issues at https://github.com/domokane/FinancePy  #
####################################################################



## Creating a CDS Contract

In [3]:
value_date = Date(20, 8, 2020)
effective_date = Date(21, 8, 2020)
maturity_date = Date(20, 6, 2025)
running_coupon = 0.05
notional = ONE_MILLION
long_protection = True

In [4]:
cds_contract = CDS(effective_date, maturity_date, running_coupon, notional, long_protection)

## Build Ibor Curve

In [5]:
settle_date = Date(24, 8, 2020)

In [6]:
dcType = DayCountTypes.ACT_360
depo1 = IborDeposit(settle_date, "1M", 0.001709, dcType)
depo2 = IborDeposit(settle_date, "2M", 0.002123, dcType)
depo3 = IborDeposit(settle_date, "3M", 0.002469, dcType)
depo4 = IborDeposit(settle_date, "6M", 0.003045, dcType)
depo5 = IborDeposit(settle_date, "12M", 0.004449, dcType)
depos = [depo1,depo2,depo3,depo4,depo5]

swapType = SwapTypes.PAY
dcType = DayCountTypes.THIRTY_E_360_ISDA
fixedFreq = FrequencyTypes.SEMI_ANNUAL
swap1 = IborSwap(settle_date,"2Y", swapType, 0.002155,fixedFreq,dcType)
swap2 = IborSwap(settle_date,"3Y", swapType, 0.002305,fixedFreq,dcType)
swap3 = IborSwap(settle_date,"4Y", swapType, 0.002665,fixedFreq,dcType)
swap4 = IborSwap(settle_date,"5Y", swapType, 0.003290,fixedFreq,dcType)
swap5 = IborSwap(settle_date,"6Y", swapType, 0.004025,fixedFreq,dcType)
swap6 = IborSwap(settle_date,"7Y", swapType, 0.004725,fixedFreq,dcType)
swap7 = IborSwap(settle_date,"8Y", swapType, 0.005430,fixedFreq,dcType)
swap8 = IborSwap(settle_date,"9Y", swapType, 0.006075,fixedFreq,dcType)
swap9 = IborSwap(settle_date,"10Y", swapType, 0.006640,fixedFreq,dcType)
swaps = [swap1,swap2,swap3,swap4,swap5,swap6,swap7,swap8,swap9]

libor_curve = IborSingleCurve(value_date, depos, [], swaps)

# Build a CDS Curve

In [7]:
cdsSpread = 0.01

In [8]:
cds1 = CDS(settle_date, "6M", cdsSpread)
cds2 = CDS(settle_date, "1Y", cdsSpread)
cds3 = CDS(settle_date, "2Y", cdsSpread)
cds4 = CDS(settle_date, "3Y", cdsSpread)
cds5 = CDS(settle_date, "4Y", cdsSpread)
cds6 = CDS(settle_date, "5Y", cdsSpread)
cds7 = CDS(settle_date, "7Y", cdsSpread)
cds8 = CDS(settle_date, "10Y", cdsSpread)

In [9]:
cdss = [cds1, cds2, cds3, cds4, cds5, cds6, cds7, cds8]

In [10]:
recovery_rate = 0.40

In [11]:
issuer_curve = CDSCurve(value_date, cdss, libor_curve, recovery_rate)

In [12]:
print(issuer_curve)

OBJECT TYPE: CDSCurve
TIME,SURVIVAL_PROBABILITY
 0.0000000,  1.0000000
 0.5808219,  0.9901922
 1.0849315,  0.9817979
 2.0849315,  0.9986272
 3.0849315,  0.9818007
 4.0876712,  0.9651915
 5.0876712,  0.9490150
 7.0876712,  0.9174925
10.0904110,  0.8721194


In [14]:
cds_contract.print_payments(value_date, issuer_curve) 

PAYMENT_DATE      YEAR_FRAC      FLOW           DF       SURV_PROB      NPV
    21-SEP-2020   0.252778     12638.89     0.999848     0.998513     12618.18
    21-DEC-2020   0.252778     12638.89     0.999079     0.994298     12555.25
    22-MAR-2021   0.252778     12638.89     0.998005     0.990101     12488.80
    21-JUN-2021   0.252778     12638.89     0.996528     0.985941     12417.93
    20-SEP-2021   0.252778     12638.89     0.995505     0.981798     12353.06
    20-DEC-2021   0.252778     12638.89     0.995554     0.985967     12406.12
    21-MAR-2022   0.252778     12638.89     0.995603     0.990154     12459.42
    20-JUN-2022   0.252778     12638.89     0.995652     0.994358     12512.94
    20-SEP-2022   0.255556     12777.78     0.995495     0.998627     12702.75
    20-DEC-2022   0.252778     12638.89     0.994848     0.994405     12503.43
    20-MAR-2023   0.250000     12500.00     0.994209     0.990247     12306.41
    20-JUN-2023   0.255556     12777.78     0.993556   

# Valuation

In [15]:
spd = cds_contract.par_spread(settle_date, issuer_curve, recovery_rate) * 10000.0
print("FAIR CDS SPREAD %10.5f bp"% spd)

FAIR CDS SPREAD   99.08198 bp


In [16]:
cds_contract.value_fast_approx(settle_date, 0.004, 0.01, 0.40, 0.40)

(-195533.6454106767, -187200.31207734335, 533.6950180429558, 44.32197353540687)

In [17]:
v = cds_contract.value(settle_date, issuer_curve, recovery_rate)

In [18]:
dirty_pv = v['dirty_pv'] 
clean_pv = v['clean_pv']

In [19]:
print("DIRTY VALUE  %12.2f"% dirty_pv)
print("CLEAN VALUE %12.2f"% clean_pv)

DIRTY VALUE    -200072.73
CLEAN VALUE   -191739.40


In [20]:
cleanp = cds_contract.clean_price(settle_date, issuer_curve, recovery_rate)
print("CLEAN PRICE %12.6f"% cleanp)

CLEAN PRICE   119.173940


In [21]:
accrued_days = cds_contract.accrued_days()
print("ACCRUED_DAYS", accrued_days)

ACCRUED_DAYS 60.0


In [22]:
accrued_interest = cds_contract.accrued_interest()
print("ACCRUED_COUPON", accrued_interest)

ACCRUED_COUPON -8333.333333333334


In [23]:
prot_pv = cds_contract.protection_leg_pv(settle_date, issuer_curve, recovery_rate)
print("PROTECTION_PV", prot_pv)

PROTECTION_PV 47386.047104632395


In [24]:
premPV = cds_contract.premium_leg_pv(settle_date, issuer_curve, recovery_rate)
print("PREMIUM_PV", premPV)

PREMIUM_PV 247458.7795293185


In [25]:
cds_contract.risky_pv01(settle_date, issuer_curve)

{'dirty_rpv01': 4.9491755905863695, 'clean_rpv01': 4.7825089239197025}

## Risk Measures

In [26]:
cds_contract.credit_dv01(settle_date, issuer_curve, recovery_rate)

507.9938544575998

In [27]:
cds_contract.interest_dv01(settle_date, issuer_curve, recovery_rate)

48.10582343424903

Copyright (c) 2020 Dominic O'Kane